In [ ]:
url = 'https://files.cooperative-robotics.com/intermediate_representation/robotics_knowledge_book_of_future/driving/'








path = '/home/adnan/derp/intermediate_representation/robotics_knowledge_book_of_future/driving'


In [ ]:
path

In [1]:
! uv pip install browser-use

Resolved 125 packages in 925ms                                       
Prepared 76 packages in 1.57s                                            
Uninstalled 5 packages in 33ms
Installed 80 packages in 47msentation-langchain==0.36.1     
 + argparse==1.4.0
 + backoff==2.2.1
 + boto3==1.36.2
 + botocore==1.36.2
 + browser-use==0.1.25
 + cachetools==5.5.0
 + courlan==1.3.2
 + dateparser==1.2.0
 + deprecated==1.2.15
 + filetype==1.2.0
 + fireworks-ai==0.15.11
 + google-ai-generativelanguage==0.6.10
 + google-api-core==2.24.0
 + google-api-python-client==2.159.0
 + google-auth==2.37.0
 + google-auth-httplib2==0.2.0
 + google-generativeai==0.8.3
 + googleapis-common-protos==1.66.0
 + grpcio==1.69.0
 + grpcio-status==1.69.0
 + html2text==2024.2.26
 + htmldate==1.9.3
 + httplib2==0.22.0
 - httpx==0.28.1
 + httpx==0.27.2
 + httpx-sse==0.4.0
 + httpx-ws==0.7.1
 + jmespath==1.0.1
 + justext==3.0.1
 - langchain==0.2.17
 + langchain==0.3.14
 - langchain-anthropic==0.1.23
 + langchain-anthropic==0.3.